In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, count, round as spark_round 

In [20]:
class MockDBUtils:
    class Widgets:
        def __init__(self):
            self.values = {}

        def text(self, name, default_value=""):
            self.values[name] = default_value

        def get(self, name):
            return self.values.get(name, "")

    def __init__(self):
        self.widgets = self.Widgets()


In [21]:
dbutils = MockDBUtils()
dbutils.widgets.text("catalog", "citibike_dev")  # ✅ default fallback
catalog = dbutils.widgets.get("catalog") or "citibike_dev"

In [22]:
spark = SparkSession.builder \
    .appName("Citibike Data") \
    .getOrCreate()

In [23]:
df = spark.read.table(f"{catalog}.`02_silver`.jc_citibike")

In [24]:

df = df \
    .groupBy("trip_start_date", "start_station_name") \
    .agg(
        spark_round(avg("trip_duration_mins"), 2).alias("avg_trip_duration_mins"),
        count("ride_id").alias("total_trips")
    )

In [9]:
df.write.\
    mode("overwrite").\
    option("overwriteSchema", "true").\
    saveAsTable(f"{catalog}.`03_gold`.daily_station_performance")